In [450]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
import findspark
findspark.init()

In [451]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Python Spark SQL basic example") \
    .config("spark.driver.maxResultSize", "1g") \
    .getOrCreate()

In [452]:
df=spark.read.csv('datasets/canadadataset.csv',header=True)
df.show()

+----------+---------+---------+-------------+----------+---------------+--------------+----------+-----------+-------+--------------------+-----------+
|visit_high|visit_low|visit_num|   event_list|hit_source|post_visit_high|post_visit_low|        ds|exclude_hit| prop70|              prop58|     prop46|
+----------+---------+---------+-------------+----------+---------------+--------------+----------+-----------+-------+--------------------+-----------+
|         0|        0|       35| 108,11,125,1|         2|              0|             0|01-07-2023|          0|    web|productPage|pageV...|     search|
|         0|        0|       35| 108,11,125,2|         2|              0|             0|02-07-2023|          0|andriod|productPage|pageV...|   homepage|
|         0|        0|       35| 108,11,125,3|         2|              0|             0|03-07-2023|          0|    ios|productPage|pageV...|productPage|
|         0|        0|       35| 108,11,125,4|         2|              0|         

In [453]:
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))
df2=df_new1.where((df_new1.ds).between('01-07-2023','31-07-2023'))
df3=df2.withColumn("ATC_RATE",split(col("event_list"),',').getItem(1))
        
#df22=df_new1.withColumn("ATC_RATE",col("event_list")==11).filter(col("ATC_RATE")==True)
#df22=df_new1.withColumn("ATC_RATE",df_new1.ds.between('01-07-2023','31-07-2023')&(df_new1.exclude_hit=='1')&(df_new1.event_list=='12'))  
#df3=df22.select(countDistinct(col("session_cnt")).alias("distn_cnt"))
df4=df3.agg(countDistinct(col("session_cnt")).alias("dst_cnt"))
df4.show(truncate=False)

+-------+
|dst_cnt|
+-------+
|3      |
+-------+



In [454]:
df3=df2.withColumn("ATC_RATE",split(col("event_list"),',').getItem(0)).select("ATC_RATE")
       
df3.show()

+--------+
|ATC_RATE|
+--------+
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
|     108|
+--------+
only showing top 20 rows



In [455]:
# df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
#          .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
# df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))\
# #df_new1.drop_duplicates(df_new1.session_cnt).show()
# #df_new1.show()

# df3=df_new1.select(countDistinct("session_cnt").alias("distn_cnt"),)
# df3.show(truncate=False)

In [456]:
df2=df_new1.where((df_new1.ds).between('01-07-2023','31-07-2023')&(df_new1.exclude_hit=='0')&(df_new1.event_list=='12'))
df2.show()

+----------+---------+---------+----------+----------+---------------+--------------+---+-----------+------+------+------+---------------+----------------+-----------+
|visit_high|visit_low|visit_num|event_list|hit_source|post_visit_high|post_visit_low| ds|exclude_hit|prop70|prop58|prop46|visit_hi_id_new|visit_low_id_new|session_cnt|
+----------+---------+---------+----------+----------+---------------+--------------+---+-----------+------+------+------+---------------+----------------+-----------+
+----------+---------+---------+----------+----------+---------------+--------------+---+-----------+------+------+------+---------------+----------------+-----------+



In [457]:
df2=df_new1.where(col("ds").between('01-07-2023','05-07-2023')).select("ds")
df2.show()

+----------+
|        ds|
+----------+
|01-07-2023|
|02-07-2023|
|03-07-2023|
|04-07-2023|
|05-07-2023|
|01-08-2023|
|02-08-2023|
|03-08-2023|
|04-08-2023|
|01-09-2023|
|02-09-2023|
|03-09-2023|
|04-09-2023|
|01-10-2023|
|02-10-2023|
|03-10-2023|
|04-10-2023|
+----------+



In [458]:
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END")) 
#df22=df_new1.withColumn("ATC_RATE",col("event_list")==11).filter(col("ATC_RATE")==True)
#df22=df_new1.withColumn("ATC_RATE",df_new1.ds.between('01-07-2023','31-07-2023')&(df_new1.exclude_hit=='0')&(df_new1.event_list=='11'))  
#df22=df2.withColumn("ATC_RATE",col("event_list")==12)
#df3=df22.select(countDistinct(col("session_cnt")).alias("distn_cnt"))
#df3.show(truncate=False)


In [557]:
df22=df_new1.filter(col("ds")>="01-07-2023" & col("ds")<="31-07-2023")
#df33=df22.groupby("visit_date").agg(countDistinct(col("session_cnt")))
df22.show()

Py4JError: An error occurred while calling o4952.and. Trace:
py4j.Py4JException: Method and([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [539]:
from pyspark.sql import functions as F
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))\
        .withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))
df3=df_new.where("ds between '01-07-2023' and '31-07-2023' and array_contains(split(event_list,','),'108')") \
           .agg(countDistinct(col("session_cnt")).alias("dsnt_cnt")) \
                   
df3.show()

+--------+
|dsnt_cnt|
+--------+
|       3|
+--------+



In [540]:
from pyspark.sql import functions as F
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))
df6=df_new1.where("ds between '01-07-2023' and '31-07-2023'")
      
df7 =df6.withColumn("ATC_RATE",array_contains(split("event_list",","),'108')).filter(col("ATC_RATE")=='true')
df8=df7.agg(countDistinct(col("session_cnt")).alias("dsnt_cnt"))
                   
df8.select("dsnt_cnt").show()

+--------+
|dsnt_cnt|
+--------+
|       3|
+--------+



In [460]:
dff=df.withColumn("pp58", substring('prop58',0,11)).where(col("pp58")=="productPage").groupBy('pp58').count()
df11=dff.where(col("pp58")=="productPage").groupBy('pp58').count()
df11.show()

+-----------+-----+
|       pp58|count|
+-----------+-----+
|productPage|    1|
+-----------+-----+



In [461]:
df7 =df6.withColumn("ATC_RATE",array_contains(split("event_list",","),'108')) \
        .withColumn("ATC_R",split("event_list",',').getItem(0))
df7.select("ATC_RATE","ATC_R").show()

+--------+-----+
|ATC_RATE|ATC_R|
+--------+-----+
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
|    true|  108|
+--------+-----+
only showing top 20 rows



In [462]:
dff=df.withColumn("pp58", substring('prop58',0,11))
dff.select('pp58').show()

+-----------+
|       pp58|
+-----------+
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
+-----------+
only showing top 20 rows



In [482]:
df12=dff.filter(col("prop58").contains("productPage"))
df12.select("prop58").show()

+--------------------+
|              prop58|
+--------------------+
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
|productPage|pageV...|
+--------------------+
only showing top 20 rows



In [485]:
df13=dff.filter(col("prop58").like("%productPage%"))


In [524]:
#dff.filter("prop58 like '%productPage%'")
dff.select("visit_high","visit_low","prop70","prop58").where(col("prop58").contains("productPage")).where(col("prop70").like("%web%")).show(truncate=False)

+----------+---------+------+----------------------------+
|visit_high|visit_low|prop70|prop58                      |
+----------+---------+------+----------------------------+
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClick|
|0         |0        |web   |productPage|pageView|onClic

In [532]:
df14=dff.withColumn("pp59",explode(split(col("prop58"),'\\|'))).filter(col("pp59")=="productPage")
df14.select("pp59").show()

+-----------+
|       pp59|
+-----------+
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
|productPage|
+-----------+
only showing top 20 rows



In [538]:
df15=dff.withColumn("evn",explode(split(col("event_list"),','))).filter(col("evn")=="108")
df15.select("evn").show()

+---+
|evn|
+---+
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
|108|
+---+
only showing top 20 rows



In [567]:
from pyspark.sql import functions as F
df_new=df.withColumn("visit_hi_id_new",expr("CASE WHEN visit_high='0' or visit_high IS NULL THEN post_visit_high ELSE Visit_high END"))\
         .withColumn("visit_low_id_new",expr("CASE WHEN visit_low='0' or visit_low IS NULL THEN post_visit_low ELSE Visit_low END"))
df_new1=df_new.withColumn("session_cnt", expr("CASE WHEN hit_source NOT IN ('5','7','8','9') then concat(visit_hi_id_new,'-',visit_low_id_new,'-',visit_num) ELSE NULL END"))
df6=df_new1.where("ds between '01-07-2023' and '31-07-2023'")
      
ATC_RATE_CNT_num =df6.withColumn("ATC_RATE_CNT",explode(split(col("event_list"),','))).filter((col("ATC_RATE_CNT")=="10") & (col("prop70")=="web"))
df8=ATC_RATE_CNT_num.agg(countDistinct("session_cnt").alias("ATC_RATE_CNT_num"))
df10=df6.agg(countDistinct("session_cnt").alias("ATC_RATE_CNT_DEN"))

Site_bounce_rate=df6.withColumn("Site_bounce_rate",explode(split(col("prop58"),'\\|'))).filter(col("Site_bounce_rate")=="productPage")
df9=Site_bounce_rate.agg(countDistinct("session_cnt").alias("Site_bounce_rate"))

df11=df8.join(df9).join(df10).show(truncate=False)

+----------------+----------------+----------------+
|ATC_RATE_CNT_num|Site_bounce_rate|ATC_RATE_CNT_DEN|
+----------------+----------------+----------------+
|1               |3               |3               |
+----------------+----------------+----------------+

